DATA:
Sales (Training Data),
Transactions, &
Events

OUTPUT: EDA & Sales Forcasting using seasonaility


Import modules and setup dependacinesd

In [125]:
# BASE
# ------------------------------------------------------
import numpy as np
import pandas as pd
import os
import gc
import time
import warnings
from pathlib import Path
#!pip install xgboost
import sklearn
from sklearn import linear_model
from sklearn.linear_model  import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit 
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
pd.set_option('display.max_colwidth', None)
import statsmodels.api as sm

import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.statespace.sarimax import SARIMAX


from pandas import datetime
from pandas import read_csv
from pandas import DataFrame
from statsmodels.tsa.arima.model import ARIMA
from matplotlib import pyplot

# DATA VISUALIZATION
# ------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# CONFIGURATIONS
# ------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings('ignore')
folder = 'C:/Users/eflanag/OneDrive - FTI Consulting/Documents/Projects/Ibex_EF/IbexV2/ibexgit/kaggle-store-sales/data/'

In [126]:
# Define Gloabl Path Variables
# ------------------------------------------------------
current_directory = os.getcwd()
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))

print("Parent directory:", parent_directory)

DATA_PATH = Path(parent_directory)
RAW_PATH = DATA_PATH / 'raw'
TRAIN_PATH = RAW_PATH / 'train.csv'
TEST_PATH = RAW_PATH / 'test.csv'
DATA_PATH



Parent directory: c:\Users\eflanag\OneDrive - FTI Consulting\Documents\Projects\Ibex_EF\IbexV2\ibexgit\kaggle-store-sales


WindowsPath('c:/Users/eflanag/OneDrive - FTI Consulting/Documents/Projects/Ibex_EF/IbexV2/ibexgit/kaggle-store-sales')

In [127]:
import importlib.util

# Specify the absolute path to the data_setup.py file
data_setup_path = '../src/data_setup.py'

# Load the module from the file
spec = importlib.util.spec_from_file_location('data_setup', data_setup_path)
data_setup = importlib.util.module_from_spec(spec)
spec.loader.exec_module(data_setup)

d = data_setup.get_data()
metrics_df = pd.DataFrame(columns=['Model', 'family_encoded', 'cols', 'MAE', 'MSE', 'RMSE','RMSLE', 'R2'])


Parent directory: c:\Users\eflanag\OneDrive - FTI Consulting\Documents\Projects\Ibex_EF\IbexV2\ibexgit\kaggle-store-sales
(3029400, 81)
complete


Modeling v0.1 - Sales forcasting using seasonailtiy

In [128]:
# Resample to reduce data for dev
dFull = d
#d = dFull.sample(frac=0.1, replace=True, random_state=1)
d.shape
d

,id,date,store_nbr,family,sales,onpromotion,test/train,city,state,type,cluster,events_Black_Friday,events_Cyber_Monday,events_Dia_de_la_Madre,events_Futbol,events_Terremoto_Manabi,holiday_national_binary,holiday_local_binary,holiday_regional_binary,national_independence,local_cantonizacio,local_fundacion,local_independencia,holiday_national_Batalla_de_Pichincha,holiday_national_Carnaval,holiday_national_Dia_de_Difuntos,holiday_national_Dia_de_la_Madre,holiday_national_Dia_del_Trabajo,holiday_national_Independencia_de_Cuenca,holiday_national_Independencia_de_Guayaquil,holiday_national_Navidad,holiday_national_Primer_Grito_de_Independencia,holiday_national_Primer_dia_del_ano,holiday_national_Viernes_Santo,holiday_regional_Provincializacion_Santa_Elena,holiday_regional_Provincializacion_de_Cotopaxi,holiday_regional_Provincializacion_de_Imbabura,holiday_regional_Provincializacion_de_Santo_Domingo,holiday_local_Cantonizacion_de_Cayambe,holiday_local_Cantonizacion_de_El_Carmen,holiday_local_Cantonizacion_de_Guaranda,holiday_local_Cantonizacion_de_Latacunga,holiday_local_Cantonizacion_de_Libertad,holiday_local_Cantonizacion_de_Quevedo,holiday_local_Cantonizacion_de_Riobamba,holiday_local_Cantonizacion_de_Salinas,holiday_local_Cantonizacion_del_Puyo,holiday_local_Fundacion_de_Ambato,holiday_local_Fundacion_de_Cuenca,holiday_local_Fundacion_de_Esmeraldas,holiday_local_Fundacion_de_Guayaquil,holiday_local_Fundacion_de_Ibarra,holiday_local_Fundacion_de_Loja,holiday_local_Fundacion_de_Machala,holiday_local_Fundacion_de_Manta,holiday_local_Fundacion_de_Quito,holiday_local_Fundacion_de_Riobamba,holiday_local_Fundacion_de_Santo_Domingo,holiday_local_Independencia_de_Ambato,holiday_local_Independencia_de_Guaranda,holiday_local_Independencia_de_Latacunga,month,day_of_month,day_of_year,week_of_month,week_of_year,day_of_week,year,is_wknd,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season,workday,wageday,dcoilwtico,dcoilwtico_interpolated
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0.00,train,Quito,Pichincha,D,13,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,NaN,93.14
1,1,2013-01-01,1,BABY CARE,0.00,0.00,train,Quito,Pichincha,D,13,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,NaN,93.14
2,2,2013-01-01,1,BEAUTY,0.00,0.00,train,Quito,Pichincha,D,13,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,NaN,93.14
3,3,2013-01-01,1,BEVERAGES,0.00,0.00,train,Quito,Pichincha,D,13,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,NaN,93.14
4,4,2013-01-01,1,BOOKS,0.00,0.00,train,Quito,Pichincha,D,13,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,2,2013,0,1,1,0,1,0,1,0,0,0,0,NaN,93.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1.00,test,Quito,Pichincha,B,6,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,31,243,5,35,4,2017,0,3,0,1,0,0,0,0,2,1,1,47.26,46.61
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0.00,test,Quito,Pichincha,B,6,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,31,243,5,35,4,2017,0,3,0,1,0,0,0,0,2,1,1,47.26,46.61
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1.00,test,Quito,Pichincha,B,6,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [129]:
# preform test train split
cols = ['test/train' ,'sales', 'store_nbr', 'type', 'family', 'date', 'month','day_of_month', 'week_of_year','day_of_week', 'dcoilwtico_interpolated']
#cols = ['test/train' ,'sales', 'family', 'date', 'month','day_of_month', 'week_of_year','day_of_week', 'type', 'dcoilwtico_interpolated']

dataTrainTest = d[cols]

dataTrain = dataTrainTest[dataTrainTest['test/train'] ==  'train'].reset_index(drop=True)
dataTest = dataTrainTest[dataTrainTest['test/train'] ==  'test'].reset_index(drop=True)

dataTrain

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
dataTrain["family_encoded"] = encoder.fit_transform(dataTrain["family"])
dataTrain.drop('family', axis=1, inplace=True)

dataTrain["type"]
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
dataTrain["type_encoded"] = encoder.fit_transform(dataTrain["type"])
dataTrain.drop('type', axis=1, inplace=True)

train_ = (dataTrain)
test_ = (dataTest)

#train_, test_ = train_test_split(dataTrain.dropna().reset_index(drop=True), test_size=0.2, random_state=42)

del d
gc.collect()


8

In [130]:
#dataTrain = dataTrain.iloc[200000:]#[29229395:-1]
#dataTrain = dataTrain.reset_index(drop=True)
#dataTrain.shape
dataTrain

,test/train,sales,store_nbr,date,month,day_of_month,week_of_year,day_of_week,dcoilwtico_interpolated,family_encoded,type_encoded
0,train,0.00,1,2013-01-01,1,1,1,2,93.14,0,3
1,train,0.00,1,2013-01-01,1,1,1,2,93.14,1,3
2,train,0.00,1,2013-01-01,1,1,1,2,93.14,2,3
3,train,0.00,1,2013-01-01,1,1,1,2,93.14,3,3
4,train,0.00,1,2013-01-01,1,1,1,2,93.14,4,3
...,...,...,...,...,...,...,...,...,...,...,...
3000883,train,438.13,9,2017-08-15,8,15,33,2,47.32,28,1
3000884,train,154.55,9,2017-08-15,8,15,33,2,47.32,29,1
3000885,train,2419.73,9,2017-08-15,8,15,33,2,47.32,30,1
3000886,train,121.00,9,2017-08-15,8,15,33,2,47.32,31,1


In [131]:
#dataTrain = dataTrain.reset_index(drop=True)
#dataTrain
dataTrain.dtypes

test/train                       category
sales                             float32
store_nbr                            int8
date                       datetime64[ns]
month                                int8
day_of_month                         int8
week_of_year                         int8
day_of_week                          int8
dcoilwtico_interpolated           float64
family_encoded                      int32
type_encoded                        int32
dtype: object

Next steps - Current leading RMSLE is 0.3

How can we improve our modeling? Utilise more features? Oil & Events. In general how does oil prices affect sales of differnent categories?

In [132]:
dataTrain = dataTrain[dataTrain['family_encoded'] == 28 ]
#dataTrain

In [133]:
cols_out = list(dataTrain.columns)
cols_out.append('y_pred')

In [134]:
dataTrain = dataTrain.sort_values(['family_encoded','date', 'type_encoded'])

dataTrain_out = pd.DataFrame(columns = cols_out)
metrics_df = pd.DataFrame(columns=['Model', 'cols', 'MAE', 'MSE', 'RMSE','RMSLE', 'R2'])
global y_pred_ARIMA

for family in dataTrain['family_encoded'].unique():
    #y_pred_ARIMA
    print(family)
    dataTrainFamily =  dataTrain[dataTrain['family_encoded'] == family ]

    dataTrainFamily = dataTrainFamily.iloc[int(84875):]#[29229395:-1]
        
    # Find the last month in the 'date' column
    last_month = dataTrainFamily['date'].max().to_period('M')

    # Calculate the start date for the test data
    start_date = last_month.start_time + pd.DateOffset(days=1)

    # Filter the DataFrame based on the dynamic timeframe
    train_ = dataTrainFamily[dataTrainFamily['date'] < start_date]
    test_ = dataTrainFamily[dataTrainFamily['date'] >= start_date]

    series = train_.copy()
    #series = series.sort_values(['date','family_encoded'])
    series.set_index('date', inplace=True)
    series
    series.index = series.index.to_period('M')


    test_inner = test_.copy()
    #test_inner = test_inner.sort_values(['date','family_encoded'])
    test_inner.set_index('date', inplace=True)
    test_inner
    test_inner.index = test_inner.index.to_period('M')

    test_exog = test_inner.copy()
    test_exog = test_exog.drop(['test/train',  'sales'] , axis=1)
    exogenous = series.drop(['test/train',  'sales'] , axis=1)

    test_ARIMA = test_inner['sales']



    def fit_sarimax(data,exogenous):
        # Fit SARIMAX model to the subset of data
        so = (1, 0, 1, 12)
        model = SARIMAX(data, order=(1, 0, 4), exog=exogenous, seasonal_order=(so))
        model_fit = model.fit()
        return model_fit

    pdqm = [
    (1, 0, 1, 12)
    #,(2, 1, 2, 12),
    #(0, 1, 1, 7),
    #(1, 0, 0, 4),
    #(0, 0, 1, 24)
    ]

    for so in pdqm:
        #for i, (train_index, test_index) in enumerate(tscv.split(train_.copy())):
        #print(f"pdqm {so}:")

        exogenous['dcoilwtico_interpolated'] = exogenous['dcoilwtico_interpolated'].astype('int8')
        # Convert categorical columns to categorical data type
        exogenous['family_encoded'] = pd.Categorical(exogenous['family_encoded'])
        exogenous['type_encoded'] = pd.Categorical(exogenous['type_encoded'])
        exogenous['month'] = pd.Categorical(exogenous['month'])
        exogenous['day_of_month'] = pd.Categorical(exogenous['day_of_month'])
        exogenous['week_of_year'] = pd.Categorical(exogenous['week_of_year'])
        exogenous['day_of_week'] = pd.Categorical(exogenous['day_of_week'])

        series['sales'] = pd.to_numeric(series['sales'], downcast='integer')
        test_ARIMA = pd.to_numeric(test_ARIMA, downcast='integer')

        test_exog['dcoilwtico_interpolated'] = test_exog['dcoilwtico_interpolated'].astype('int8')
        # Convert categorical columns to categorical data type
        test_exog['family_encoded'] = pd.Categorical(test_exog['family_encoded'])
        test_exog['type_encoded'] = pd.Categorical(test_exog['type_encoded'])
        test_exog['month'] = pd.Categorical(test_exog['month'])
        test_exog['day_of_month'] = pd.Categorical(test_exog['day_of_month'])
        test_exog['week_of_year'] = pd.Categorical(test_exog['week_of_year'])
        test_exog['day_of_week'] = pd.Categorical(test_exog['day_of_week'])        
    


        # Get the memory usage of each column
        memory_usage = exogenous.memory_usage(deep=True)
        #print("\nMemory Usage:")
        #print(memory_usage)
        
        # Calculate memory usage in bytes
        memory_bytes = exogenous.memory_usage(deep=True).sum()
        # Convert to gigabytes (GB)
        memory_gb = memory_bytes / (1024**3)
        # Print the size in GB
        print(f"Size of data: {memory_gb:.2f} GB")

        start_time = time.time()
        #model = ARIMA(series, order=(1,0,4), exog=exogenous )


        model = SARIMAX(series['sales'], order=(1, 0, 4), exog=exogenous, seasonal_order=(so))
        model_fit = model.fit()
        del model


        y_pred_ARIMA = model_fit.forecast(steps=len(test_inner['sales']), exog= test_exog)

        test_['y_pred_raw'] = y_pred_ARIMA.values

        y_pred_ARIMA[y_pred_ARIMA < 0] = 0


        if (y_pred_ARIMA < 0).any():
            rmsle = 9999999

        else:    
            #print((test_ARIMA.shape),(y_pred_ARIMA.shape))
            #print(type(test_ARIMA),type(y_pred_ARIMA.shape))
            rmsle = np.sqrt(mean_squared_log_error(test_ARIMA, y_pred_ARIMA))

        # Evaluate the model
        mae = mean_absolute_error(test_ARIMA, y_pred_ARIMA)
        mse = mean_squared_error(test_ARIMA, y_pred_ARIMA)
        rmse = np.sqrt(mse)

        r2 = r2_score(test_ARIMA, y_pred_ARIMA)

        print(f'MAE: {mae:.2f}')
        print(f'MSE: {mse:.2f}')
        print(f'RMSE: {rmse:.2f}')
        print(f'RMSLE: {rmsle:.2f}')
        print(f'R2 score: {r2:.2f}')


        test_['y_pred'] = y_pred_ARIMA.values
        
        dataTrain_out = dataTrain_out.append(test_)

        metrics_df = metrics_df.append({'Model': 'SARIMAX', 'family_encoded':family , 'cols': list(train_.columns),  'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'RMSLE': rmsle, 'R2': r2}, ignore_index=True)
        print('rolling mean RMSLE',metrics_df['RMSLE'].mean())
        del y_pred_ARIMA, test_exog, series, exogenous
        gc.collect()
        
        #print(dataTrainFamily)
dataTrain_out



28
Size of data: 0.00 GB
MAE: 279.16
MSE: 126366.13
RMSE: 355.48
RMSLE: 1.06
R2 score: 0.02
rolling mean RMSLE 1.0564673628134784


,test/train,sales,store_nbr,date,month,day_of_month,week_of_year,day_of_week,dcoilwtico_interpolated,family_encoded,type_encoded,y_pred,y_pred_raw
2166874,train,1291.41,44,2017-08-02,8,2,31,3,49.27,28,0,828.32,828.32
2222446,train,1298.33,45,2017-08-02,8,2,31,3,49.27,28,0,808.50,808.50
2278018,train,834.70,46,2017-08-02,8,2,31,3,49.27,28,0,739.75,739.75
2333590,train,1132.24,47,2017-08-02,8,2,31,3,49.27,28,0,755.70,755.70
2389162,train,908.72,48,2017-08-02,8,2,31,3,49.27,28,0,707.76,707.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2945311,train,548.98,8,2017-08-15,8,15,33,2,47.32,28,3,391.41,391.41
1167007,train,194.75,28,2017-08-15,8,15,33,2,47.32,28,4,110.01,110.01
1222579,train,165.98,29,2017-08-15,8,15,33,2,47.32,28,4,103.04,103.04
1667155,train,191.68,36,2017-08-15,8,15,33,2,47.32,28,4,54.23,54.23


In [135]:
dataTrain_out

,test/train,sales,store_nbr,date,month,day_of_month,week_of_year,day_of_week,dcoilwtico_interpolated,family_encoded,type_encoded,y_pred,y_pred_raw
2166874,train,1291.41,44,2017-08-02,8,2,31,3,49.27,28,0,828.32,828.32
2222446,train,1298.33,45,2017-08-02,8,2,31,3,49.27,28,0,808.50,808.50
2278018,train,834.70,46,2017-08-02,8,2,31,3,49.27,28,0,739.75,739.75
2333590,train,1132.24,47,2017-08-02,8,2,31,3,49.27,28,0,755.70,755.70
2389162,train,908.72,48,2017-08-02,8,2,31,3,49.27,28,0,707.76,707.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2945311,train,548.98,8,2017-08-15,8,15,33,2,47.32,28,3,391.41,391.41
1167007,train,194.75,28,2017-08-15,8,15,33,2,47.32,28,4,110.01,110.01
1222579,train,165.98,29,2017-08-15,8,15,33,2,47.32,28,4,103.04,103.04
1667155,train,191.68,36,2017-08-15,8,15,33,2,47.32,28,4,54.23,54.23


In [124]:
metrics_df.sort_values(by='RMSLE')

,Model,cols,MAE,MSE,RMSE,RMSLE,R2,family_encoded
0,SARIMAX,"[test/train, sales, store_nbr, date, month, day_of_month, week_of_year, day_of_week, dcoilwtico_interpolated, family_encoded, type_encoded]",279.16,126366.13,355.48,1.06,0.02,28.00
